In [1]:
import os
from pathlib import Path
import glob
import pandas as pd
from nvita.utils import open_json
from nvita.attacks.utils import (append_result_to_csv_file,
                                 check_result_file_path,
                                 create_empty_result_csv_file)

D:\Programming\IDE\Anaconda\envs\hons\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_ROOT = Path(os.getcwd()).parent.absolute()
PATH_NTRE = os.path.join(PATH_ROOT, "results", "non_targeted_results")
PATH_TARE = os.path.join(PATH_ROOT, "results", "targeted_results")
PATH_EXPS = os.path.join(PATH_ROOT, "experiments")

In [3]:
my_metadata = open_json(os.path.join(
    PATH_ROOT, "experiments", "metadata.json"))
print(my_metadata)

{'data': ['Electricity', 'NZTemp', 'CNYExch', 'Oil'], 'seeds': ['2210', '9999', '58361', '789789', '1111111'], 'y_col_name': ['Consumption', 'Auckland', 'Close', 'Close'], 'window_size': ['4', '3', '7', '7'], 'models': ['CNN', 'LSTM', 'GRU', 'RF'], 'attacks': ['NOATTACK', 'BRS', 'BRNV', 'FGSM', 'BIM', 'NVITA', 'FULLVITA'], 'n_values': [1, 3, 5], 'epsilons': [0.05, 0.1, 0.15, 0.2], 'targets': ['Positive', 'Negative']}


In [4]:
class RequiredCSV:
    """
    Required CSV class
    """
    def __init__(self, df_name = None, seed = None, eps = None, model = None, attack = None, targeted = False, target_direction = None, count = 0):
        self.df_name = df_name
        self.seed = seed
        self.eps = eps
        self.model = model
        self.attack = attack
        self.targeted = targeted
        self.target_direction = target_direction
        self.count = count
        self.all_names = []
        # all_names list is used to check duplications
    def get_output_line_list(self):
        if self.targeted:
            return [self.df_name, self.seed, self.eps, self.model, self.attack, self.target_direction, self.count, str(self)]
        else:
            return [self.df_name, self.seed, self.eps, self.model, self.attack, self.count, str(self)]
        
    def __str__(self) -> str:
        if self.targeted:
            return "df_"+self.df_name+"_seed_"+str(self.seed)+"_model_"+model+"_epsilon_"+str(self.eps)+"_attack_"+attack+"_target_"+self.target_direction
        else:
            return "df_"+self.df_name+"_seed_"+str(self.seed)+"_model_"+model+"_epsilon_"+str(self.eps)+"_attack_"+attack

## Non-targeted Check

In [5]:
nont_d = {}
total_count = 0
for df in my_metadata["data"]:
    for seed in my_metadata["seeds"]:
        for eps in my_metadata["epsilons"]:
            for model in my_metadata["models"]:
                if model == "RF":
                    attacks = ["NOATTACK", "BRS", "Non_Targeted_BR5V", "Non_Targeted_1VITA", "Non_Targeted_3VITA", "Non_Targeted_5VITA", "FULLVITA"]
                else:
                    attacks = ["NOATTACK", "BRS", "Non_Targeted_BR5V", "FGSM", "BIM", "Non_Targeted_1VITA", "Non_Targeted_3VITA", "Non_Targeted_5VITA", "FULLVITA"]
                for attack in attacks:
                    rcsv = RequiredCSV(df, seed, eps, model, attack)
                    total_count += 1
                    nont_d[str(rcsv)] = rcsv
print("Total number of csv result files for non-targeted experiments are " + str(total_count))

Total number of csv result files for non-targeted experiments are 2720


In [6]:
for (root,dirs,files) in os.walk(PATH_NTRE, topdown=True):
    for file in (f for f in files if f.endswith(".csv")):
        file_path = Path(root, file)
        df = pd.read_csv(file_path, index_col=None, header=0)
        if df.shape[0] == 100:
            # Complete result
            suffix_ind = -5
            while file[suffix_ind] == "1":
                suffix_ind -= 1
            nont_d[file[:suffix_ind+1]].count+=1
            nont_d[file[:suffix_ind+1]].all_names.append(file)
            # Append the existing file name, it can be modified to remove duplication

In [7]:
nt_fl = ["df_name", "seed", "eps", "model", "attack", "occurrence", "full_name"]
nt_path = os.path.join(PATH_ROOT, "results", "non_targeted_check_result.csv")
create_empty_result_csv_file(nt_path, nt_fl)
for k in nont_d:
    line_list = nont_d[k].get_output_line_list()
    append_result_to_csv_file(nt_path, line_list)

# Non-targeted Check

In [8]:
t_d = {}
total_count = 0
for df in my_metadata["data"]:
    for seed in my_metadata["seeds"]:
        for eps in my_metadata["epsilons"]:
            for td in my_metadata["targets"]:
                for model in my_metadata["models"]:
                    if model == "RF":
                        attacks = ["NOATTACK", "BRS", "Targeted_BR5V", "Targeted_1VITA", "Targeted_3VITA", "Targeted_5VITA", "FULLVITA"]
                    else:
                        attacks = ["NOATTACK", "BRS", "Targeted_BR5V", "FGSM", "BIM", "Targeted_1VITA", "Targeted_3VITA", "Targeted_5VITA", "FULLVITA"]
                    for attack in attacks:
                        rcsv = RequiredCSV(df, seed, eps, model, attack, True, td)
                        total_count += 1
                        t_d[str(rcsv)] = rcsv
print("Total number of csv result files for targeted experiments are " + str(total_count))

Total number of csv result files for targeted experiments are 5440


In [9]:
for (root,dirs,files) in os.walk(PATH_TARE, topdown=True):
    for file in (f for f in files if f.endswith(".csv")):
        file_path = Path(root, file)
        df = pd.read_csv(file_path, index_col=None, header=0)
        if df.shape[0] == 100:
            # Complete result
            suffix_ind = -5
            while file[suffix_ind] == "1":
                suffix_ind -= 1
            t_d[file[:suffix_ind+1]].count+=1
            t_d[file[:suffix_ind+1]].all_names.append(file)
            # Append the existing file name, it can be modified to remove duplication

In [10]:
ta_fl = ["df_name", "seed", "eps", "model", "attack", "target_direction", "occurrence", "full_name"]
ta_path = os.path.join(PATH_ROOT, "results", "targeted_check_result.csv")
create_empty_result_csv_file(ta_path, ta_fl)
for k in t_d:
    line_list = t_d[k].get_output_line_list()
    append_result_to_csv_file(ta_path, line_list)